<h1 align="center">OpenSAFELY Service Restoration Observatory Monthly Dashboard: Eleven key measures for monitoring general practice clinical activity during COVID-19 </h1>

**This notebook accompanies [this report](https://reports.opensafely.org/reports/sro-measures/), describing trends and variation in clinical activity codes using a set of key measures indicative of overall activity to evaluate NHS service restoration throughout the COVID-19 pandemic. For details on the results presented here, please refer to the main report or to the accompanying [preprint](https://www.medrxiv.org/content/10.1101/2022.10.17.22281058v1.article-info).**

The key differences between this version and the accompanying report are:

* This analysis is run on data for patients registered at a TPP practice. This covers 40% of the population of England. For a description of the representativeness of this sample, please see our manuscript [here](https://doi.org/10.12688/wellcomeopenres.18010.1).
* Our previous analysis was restricted to adults. This analysis includes all patients, so the observed rates for some measures will be lower.
* Our previous analysis selected all patients who were registered at the start of each  month, using a rolling population. This analysis selects a static population as all patients registered as of the end of the study period or who have died during the study period.


<h3 class="details">Summary of results</h3>

These key measures demonstrated substantial changes in clinical activity throughout the COVID-19 pandemic. Six of the measures recovered to their pre-pandemic baseline within a year of the pandemic, showing a rapid, adaptive response by primary care in the midst of a global health pandemic. The remaining five measures showed a more sustained drop in activity; asthma and COPD reviews did not recover to their pre-pandemic baseline until around August 2021 and blood pressure monitoring, cardiovascular disease risk assessment and medication reviews had a sustained drop in activity that persisted up to December 2021.

<h3 class="details">Findings in context</h3>

Discussion of the specific causes and reasons for the changes in narrow measures of clinical activity we have described is best addressed through quantitative analyses that identify practices in high and low deciles to approach for targeted qualitative interviews with patients and front line staff. However we believe the following broad points may help aid interpretation. Our measures reflect only a few areas of high volume clinical activity; decreases may reflect appropriate prioritisation of other clinical activity. For example NHS Health Checks, which are used to detect early signs of high blood pressure, heart disease or type 2 diabetes, were paused during the pandemic; this is likely to explain the sustained drop in activity in cardiovascular disease risk assessment and blood pressure monitoring. However, in specific cases this may reflect changes in the style of delivery of a clinical activity, rather than the volume: for example, where patients record their own blood pressure at home since, as we have previously highlighted, home monitoring of blood pressure may not be recorded completely or consistently in GP records. In addition, not all reductions should be interpreted as problematic: as part of the COVID-19 recovery, health systems are aiming to be more resilient, responsive and sustainable; complete recovery may not always be appropriate and reductions in clinical activity across some domains may reflect rational reprioritisation of activity. Where these changes in priority have not been nationally planned, data analyses such as ours may help to rapidly identify the pragmatic changes in prioritisation being made by individual dispersed organisations or people across the healthcare ecosystem before those changes are explicitly surfaced or discussed through other mechanisms. For more detail, please see our preprint [here](https://www.medrxiv.org/content/10.1101/2022.10.17.22281058v1.article-info).

The following key measures are provided:

<ul id="docNav">

<li> <a href="#systolic_bp">Blood Pressure Monitoring</a>
<li> <a href="#qrisk2">Cardiovascular Disease 10 Year Risk Assessment</a>
<li> <a href="#cholesterol">Cholesterol Testing</a>
<li> <a href="#ALT">Liver Function Testing - Alanine Transferaminase (ALT)</a>
<li> <a href="#serum_tsh">Thyroid Testing</a>
<li> <a href="#rbc_fbc">Full Blood Count - Red Blood Cell (RBC) Testing</a>
<li> <a href="#hba1c">Glycated Haemoglobin A1c Level (HbA1c)</a>
<li> <a href="#serum_sodium">Renal Function Assessment - Sodium Testing</a>
<li> <a href="#asthma">Asthma Reviews</a>
<li> <a href="#copd">Chronic Obstrutive Pulmonary Disease (COPD) Reviews</a>
<li> <a href="#med_review">Medication Review</a>

</ul>

In [ ]:
import pandas as pd
from utilities import *
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline
%config InlineBackend.figure_format='png'

In [ ]:
%%capture --no-display

sentinel_measures = ["qrisk2", "asthma", "copd", "sodium", "cholesterol", "alt", "tsh", "rbc", 'hba1c', 'systolic_bp', 'medication_review']


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

def replace_with_correct_deciles(series):
    series = pd.Series([10, 20, 30, 40, 50, 60, 70, 80, 90, 1, 2, 3, 4, 5, 6, 7, 8, 9, 91, 92, 93, 94, 95, 96, 97, 98, 99])
    return series

def deciles_chart_ebm(
    df,
    period_column=None,
    column=None,
    title="",
    ylabel="",
    show_outer_percentiles=True,
    show_legend=True,
    ax=None,
):
    """period_column must be dates / datetimes"""
    sns.set_style("whitegrid", {"grid.color": ".9"})
    if not ax:
        fig, ax = plt.subplots(1, 1)
    
    linestyles = {
        "decile": {
            "line": "b--",
            "linewidth": 1,
            "label": "decile",
        },
        "median": {
            "line": "b-",
            "linewidth": 1.5,
            "label": "median",
        },
        "percentile": {
            "line": "b:",
            "linewidth": 0.8,
            "label": "1st-9th, 91st-99th percentile",
        },
    }
    label_seen = []
    for percentile in range(1, 100):  # plot each decile line
        data = df[df["percentile"] == percentile]
        add_label = False

        if percentile == 50:
            style = linestyles["median"]
            add_label = True
        elif show_outer_percentiles and (percentile < 10 or percentile > 90):
            style = linestyles["percentile"]
            if "percentile" not in label_seen:
                label_seen.append("percentile")
                add_label = True
        else:
            style = linestyles["decile"]
            if "decile" not in label_seen:
                label_seen.append("decile")
                add_label = True
        if add_label:
            label = style["label"]
        else:
            label = "_nolegend_"

        ax.plot(
            data[period_column],
            data[column],
            style["line"],
            linewidth=style["linewidth"],
            label=label,
        )
    ax.set_ylabel(ylabel, size=15, alpha=0.6)
    if title:
        ax.set_title(title, size=18)
    # set ymax across all subplots as largest value across dataset
    ax.set_ylim([0, df[column].max() * 1.05])
    ax.tick_params(labelsize=12)
    ax.set_xlim(
        [df[period_column].min(), df[period_column].max()]
    )  # set x axis range as full date range

    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%B %Y"))
    ax.xaxis.set_major_locator(matplotlib.dates.MonthLocator(interval=1))
    if show_legend:
        ax.legend(
            bbox_to_anchor=(1.05, 0.6),
            ncol=1,
            fontsize=12,
            borderaxespad=0.0,
            frameon=True,
        )

    # rotates and right aligns the x labels, and moves the bottom of the
    # axes up to make room for them
    plt.gcf().autofmt_xdate(rotation=90, ha="center", which="both")

    return plt

for m in sentinel_measures:
    # Load deciles and generate deciles chart
    deciles = pd.read_csv(f"../output/{m}/deciles_table_counts_per_week_per_practice.csv", parse_dates=['date'])

    # fix deciles
    deciles.loc[:, "percentile"] = deciles.groupby(by=["date"])[["percentile"]].transform(replace_with_correct_deciles)
    
    
    px = 1 / plt.rcParams["figure.dpi"]  # pixel in inches
    fig, ax = plt.subplots(
        1, 1, figsize=(800 * px, 400 * px), tight_layout=True
    )



    plot = deciles_chart_ebm(
        deciles,
        period_column="date",
        column="value",
        ylabel="rate per 1000",
        show_outer_percentiles=True,
        ax=ax,
    )
    plt.savefig(f"../output/{m}/deciles_chart_counts_per_week_per_practice.png")
    plt.close()
    

In [ ]:
%%capture
# non-displayed initial run due to gridlines bug
generate_sentinel_measure('systolic_bp', 'output')

<a id="systolic_bp"></a>
## Blood Pressure Monitoring

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/systolic-blood-pressure-qof/3572b5fb/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

A commonly-used assessment used to identify patients with hypertension or to ensure optimal treatment for those with known hypertension.  This helps ensure appropriate treatment, with the aim of reducing long term risks of complications from hypertension such as stroke, myocardial infarction and kidney disease. 

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure('systolic_bp', 'output')

<a id="qrisk2"></a>
## Cardiovascular Disease 10 year Risk Assessment

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/cvd-risk-assessment-score-qof/1adf44a5/">this codelist</a>.

<h3 class="details">What is it and why does it matter? </h3>

A commonly-used risk assessment used to identify patients with an increased risk of cardiovascular events in the next 10 years. This helps ensure appropriate treatment, with the aim of reducing long term risks of complications such as stroke or myocardial infarction. 

In [ ]:
generate_sentinel_measure('qrisk2', 'output')

<a id="cholesterol"></a>
## Cholesterol Testing

The codes used in this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/cholesterol-tests/09896c09/">Codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

A commonly-used blood test used as part of a routine cardiovascular disease 10 year risk assessment and also to identify patients with lipid disorders (e.g. familial hypercholesterolaemia). This helps ensure appropriate treatment, with the aim of reducing long term risks of complications such as stroke or myocardial infarction.

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure('cholesterol', 'output')

<a id="ALT"></a>
## Liver Function Testing - Alanine Transferaminase (ALT)

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/alanine-aminotransferase-alt-tests/2298df3e/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

An ALT blood test is one of a group of liver function tests (LFTs) which are used to detect problems with the function of the liver.  It is often used to monitor patients on medications which may affect the liver or which rely on the liver to break them down within the body. They are also tested for patients with known or suspected liver dysfunction.  

<h3 class="details">Caveats</h3>

**In a small number of places, an ALT test may NOT be included within a liver function test**. We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure('alt', 'output')

<a id="serum_tsh"></a>
## Thyroid Testing

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/thyroid-stimulating-hormone-tsh-testing/11a1abeb/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

TSH is used for the diagnosis and monitoring of hypothyroidism and hyperthyroidism, including making changes to thyroid replacement therapy dosing.

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure('tsh', 'output')

<a id="rbc_fbc"></a>
## Full Blood Count - Red Blood Cell (RBC) Testing

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/red-blood-cell-rbc-tests/576a859e/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

RBC is completed as part of a group of tests referred to as a full blood count (FBC), used to detect a variety of disorders of the blood, such as anaemia and infection.

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure('rbc', 'output')

<a id="hba1c"></a>
## Glycated Haemoglobin A1c Level (HbA1c)

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/glycated-haemoglobin-hba1c-tests/62358576/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

HbA1c is a long term indicator of diabetes control. NICE guidelines recommend that individuals with diabetes have their HbA1c measured at least twice a year. Poor diabetic control can place individuals living with diabetes at an increased risk of the complications of diabetes.

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure('hba1c', 'output')


<a id="serum_sodium"></a>
## Renal Function Assessment - Sodium Testing

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/sodium-tests-numerical-value/32bff605/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

Sodium is completed as part of a group of tests referred to as a renal profile, used to detect a variety of disorders of the kidneys. A renal profile is also often used to monitor patients on medications which may affect the kidneys or which rely on the kidneys to remove them from the body.

<h3 class="details">Caveats</h3>

We use codes which represent results reported to GPs so tests requested but not yet reported are not included. Only test results returned to GPs are included, which will usually exclude tests requested while a person is in hospital and other settings like a private clinic.

In [ ]:
generate_sentinel_measure('sodium', 'output')

<a id="asthma"></a>
## Asthma Reviews

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/asthma-annual-review-qof/33eeb7da/">this codelist</a>.

<h3 class="details">What is it and why does it matter?</h3>

The British Thoracic Society and Scottish Intercollegiate Guidelines Network on the management of asthma recommend that people with asthma receive a review of their condition at least annually. If a patient has not been reviewed, it is possible that their asthma control may have worsened, leading to a greater chance of symptoms and admission to hospital.

In [ ]:
generate_sentinel_measure('asthma', 'output')


<a id="copd"></a>
## Chronic Obstructive Pulmonary Disease (COPD) Reviews

The codes used for this measure are available in <a href="https://www.opencodelists.org/codelist/opensafely/chronic-obstructive-pulmonary-disease-copd-review-qof/01cfd170/">this codelist</a>.  

<h3 class="details">What is it and why does it matter?</h3>

It is recommended by NICE that all individuals living with COPD have an annual review with the exception of individuals living with very severe (stage 4) COPD being reviewed at least twice a year.
If a patient has not been reviewed, it is possible that their COPD control may have worsened, leading to a greater chance of symptoms and admission to hospital.


In [ ]:
generate_sentinel_measure('copd', 'output')

<a id="med_review"></a>
## Medication Reviews

The codes used in for this measure are a combination of codes available in <a href="https://www.opencodelists.org/codelist/opensafely/care-planning-medication-review-simple-reference-set-nhs-digital/61b13c39/"> this NHS Digital care planning medication review refset</a> and <a href="https://www.opencodelists.org/codelist/nhsd-primary-care-domain-refsets/medrvw_cod/20200812/">this primary care domain medication review refset</a>.
 
<h3 class="details">What is it and why does it matter?</h3>

Many medicines are used long-term and they should be reviewed regularly to ensure they are still safe, effective and appropriate.
Medication review is a broad term ranging from a notes-led review without a patient, to an in-depth Structured Medication Review with multiple appointments and follow-up. The codelist provided captures all types of reviews to give an overview of medication reviews in primary care.

In [ ]:
generate_sentinel_measure('medication_review', 'output')